Package + data import

In [43]:
import os
import sys
import pandas as pd 
import numpy as np
from datetime import datetime
import importlib
import sqlalchemy as sq
from dotenv import load_dotenv
from IPython.display import display
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import xlsxwriter
from ipywidgets import Dropdown, Button, HBox, VBox, Output, interact, Tab

In [44]:
data = pd.read_csv('Ocean_heat_content_1998_2013_Fig7.csv')
data.head()

,Analysis_domain,Reference,Analysis_interval,Avg_interval_biweekly,Heat_1988-2013_avg,Heat_1988-2013_std,Heat_1988-2013_trnd,Heat_1988-2013_trnd_sig,Heat_1982-1990_avg,Heat_1991-2000_avg,Heat_2001-2013_avg,Reference_file
0,Arctic Ocean-Coastal,Figure 7a,1988-2013,11-Jan,418.86,1.4711,71.436,99,369.47,399.29,445.30,Fig7a_Arctic.txt
1,Arctic Ocean-Coastal,Figure 7a,1988-2013,25-Jan,396.71,2.2173,66.018,99,352.82,377.95,421.27,Fig7a_Arctic.txt
2,Arctic Ocean-Coastal,Figure 7a,1988-2013,08-Feb,380.12,1.8920,59.012,95,345.47,361.07,402.76,Fig7a_Arctic.txt
3,Arctic Ocean-Coastal,Figure 7a,1988-2013,22-Feb,366.92,3.7255,51.159,95,337.55,349.33,387.23,Fig7a_Arctic.txt
4,Arctic Ocean-Coastal,Figure 7a,1988-2013,08-Mar,358.25,6.2549,45.391,90,329.41,342.87,376.75,Fig7a_Arctic.txt


Data formatting + summary statistics

In [82]:
data['Heat_1982-1990_avg'] = data['Heat_1982-1990_avg'].astype(float)
data['Heat_1991-2000_avg'] = data['Heat_1991-2000_avg'].astype(float)
data['Heat_2001-2013_avg'] = data['Heat_2001-2013_avg'].astype(float)

total_heat = data['Heat_1988-2013_avg']
heat_1982_thru_1990 = data['Heat_1982-1990_avg']
heat_1991_thru_2000 = data['Heat_1991-2000_avg']
heat_2001_thru_2013 = data['Heat_2001-2013_avg']

ocean_list = data['Analysis_domain'].unique()

In [68]:
summary = data.pivot_table(index='Analysis_domain',
                            values=['Heat_1982-1990_avg', 
                                    'Heat_1991-2000_avg', 
                                    'Heat_2001-2013_avg'],
                            aggfunc={'Heat_1982-1990_avg': [np.min, np.max, np.mean],
                                    'Heat_1991-2000_avg': [np.min, np.max, np.mean],
                                    'Heat_2001-2013_avg': [np.min, np.max, np.mean]})
summary

Heat_1982-1990_avg                      \
                                          amax    amin        mean   
Analysis_domain                                                      
Arctic Ocean-Coastal                    995.85  325.75  568.360385   
Eurasia Ocean-Coastal                   924.96  284.47  526.111923   
North America Ocean-Coastal            1053.30  359.18  602.576154   

                            Heat_1991-2000_avg                      \
                                          amax    amin        mean   
Analysis_domain                                                      
Arctic Ocean-Coastal                    985.20  340.56  576.460000   
Eurasia Ocean-Coastal                   862.13  286.05  500.801923   
North America Ocean-Coastal            1084.90  384.48  637.738846   

                            Heat_2001-2013_avg                      
                                          amax    amin        mean  
Analysis_domain                                                     
Arctic Ocean-Coastal                   1118.00  368.16  647.554231  
Eurasia Ocean-Coastal                   999.63  312.21  575.946154  
North America Ocean-Coastal            1213.90  413.48  705.554615

Heat average over time

In [55]:
fig = go.Figure()

for ocean in ocean_list:
    df = data[data['Analysis_domain'] == ocean]
    fig.add_trace(go.Scatter(x=df['Avg_interval_biweekly'], 
                             y=df['Heat_1988-2013_avg'], 
                             mode='lines+markers', 
                             name=ocean))

fig.update_layout(title='Average Ocean Heat Content --- 1988 thru 2013', 
                  xaxis_title='Year', 
                  yaxis_title='Heat Content')

fig.show()

Heat content by study interval

In [67]:
measures = ['Heat_1982-1990_avg', 'Heat_1991-2000_avg', 'Heat_2001-2013_avg']

figs = []

for ocean in ocean_list:
    df = data[data['Analysis_domain'] == ocean]
    fig = go.Figure()
    
    for measure in measures:
        fig.add_trace(go.Scatter(x=df['Avg_interval_biweekly'], 
                                 y=df[measure], 
                                 mode='lines+markers', 
                                 name=measure))
        
    fig.update_layout(title=f'{ocean} --- heat content from 1982 thru 2013 (per interval)',
                      xaxis_title='Year',
                      yaxis_title='Heat Content')
    
    figs.append(fig)

for fig in figs:
    fig.show()

Heat content by month

In [114]:
time = data['Avg_interval_biweekly'].unique()

for ocean in ocean_list:
        df = data[data['Analysis_domain'] == ocean]
        fig = go.Figure(data=go.Heatmap(z=df[measures].values,
                                        x=measures,
                                        y=time
                                        ))
        
        fig.update_layout(title=f'{ocean} --- Heat Content from 1982 thru 2013',
                         xaxis_title='Year',
                         yaxis_title='Heat Content',
                         yaxis_autorange='reversed')
        
        fig.show()

Conclusion:

--- All three pan-Arctic oceans (North American, Arctic, Eurasia) showed significant increases in heat levels (in MJ m-2) over 30-year study


--- Each ocean individually shows exclusive patterns in heat content, yet all three oceans studied show similar rising heat levels in the latest time interval measured (2001 – 2013)



--- Seasonally, heat content began to rise in April, and peaking in highest levels in September